In [ ]:
import os 

# Setup the denoiser repo 
!git clone https://github.com/YunyangZeng/TAPLoss.git --quiet
%cd TAPLoss/Demucs/denoiser

!pip uninstall -y hydra_core hydra_colorlog pystoi git+https://github.com/ludlows/python-pesq#egg=pesq six sounddevice torchaudio torch 
!pip install -q julius hydra_core==0.11.3 hydra_colorlog==0.1.4 pystoi==0.3.3 git+https://github.com/ludlows/python-pesq#egg=pesq six==1.16.0 sounddevice torchaudio==0.13.1 torch==1.13.1


In [ ]:
pip install modelscope

In [ ]:
from modelscope.pipelines import pipeline
from modelscope.utils.constant import Tasks

ans = pipeline(
    Tasks.acoustic_noise_suppression,
    model='damo/speech_frcrn_ans_cirm_16k')
result = ans('https://modelscope.cn/api/v1/models/damo/speech_frcrn_ans_cirm_16k/repo?Revision=master&FilePath=examples/speech_with_noise1.wav',
    output_path='output.wav')

In [ ]:
!python -m denoiser.audio ../../../drive/MyDrive/IDLProject/CODE/april27/src_teams_cloud_relay_low > ../../../drive/MyDrive/IDLProject/CODE/april27/low_noisy_filenames.json 
!python -m denoiser.audio ../../../drive/MyDrive/IDLProject/CODE/april27/src_teams_cloud_relay_auto > ../../../drive/MyDrive/IDLProject/CODE/april27/auto_noisy_filenames.json 


In [ ]:
# create folders
! mkdir ../../../drive/MyDrive/IDLProject/CODE/april27/baseline_enhanced_low
! mkdir ../../../drive/MyDrive/IDLProject/CODE/april27/baseline_enhanced_auto
! mkdir ../../../drive/MyDrive/IDLProject/CODE/april27/finetuned_enhanced_low
! mkdir ../../../drive/MyDrive/IDLProject/CODE/april27/finetuned_enhanced_auto

In [ ]:
# Generate enhanced files with baseline demucs model and store them into ../enhan
!python -m denoiser.enhance --dns64 --num_workers 4 --noisy_json ../../../drive/MyDrive/IDLProject/CODE/april27/low_noisy_filenames.json --out_dir ../../../drive/MyDrive/IDLProject/CODE/april27/baseline_enhanced_low
!python -m denoiser.enhance --dns64 --num_workers 4 --noisy_json ../../../drive/MyDrive/IDLProject/CODE/april27/auto_noisy_filenames.json --out_dir ../../../drive/MyDrive/IDLProject/CODE/april27/baseline_enhanced_auto

In [12]:
# Generate enhanced files with finetuned demucs model and store them into ../enhan
!python -m denoiser.enhance --model_path "../../../drive/MyDrive/IDLProject/may1/best_low05.th" --num_workers 4 --noisy_json ../../../drive/MyDrive/IDLProject/april27/low_noisy_filenames.json --out_dir ../../../drive/MyDrive/IDLProject/april27/finetuned_enhanced_low
!python -m denoiser.enhance --model_path "../../../drive/MyDrive/IDLProject/may1/best_auto.th" --num_workers 4 --noisy_json ../../../drive/MyDrive/IDLProject/april27/auto_noisy_filenames.json --out_dir ../../../drive/MyDrive/IDLProject/april27/finetuned_enhanced_auto


INFO:denoiser.pretrained:Loading model from ../../../drive/MyDrive/IDLProject/may1/best_low05.th
Traceback (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/content/TAPLoss/Demucs/denoiser/denoiser/enhance.py", line 159, in <module>
    enhance(args, local_out_dir=args.out_dir)
  File "/content/TAPLoss/Demucs/denoiser/denoiser/enhance.py", line 113, in enhance
    model = pretrained.get_model(args).to(args.device)
  File "/content/TAPLoss/Demucs/denoiser/denoiser/pretrained.py", line 66, in get_model
    pkg = torch.load(args.model_path, 'cpu')
  File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 777, in load
    with _open_zipfile_reader(opened_file) as opened_zipfile:
  File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 282, in __init__
    super

In [ ]:
# Remove noisy files from enhanced folder
! rm -rf ../../../drive/MyDrive/IDLProject/april27/baseline_enhanced_low/*_noisy.wav
! rm -rf ../../../drive/MyDrive/IDLProject/april27/baseline_enhanced_auto/*_noisy.wav

In [ ]:
# Remove noisy files from enhanced folder
! rm -rf ../../../drive/MyDrive/IDLProject/april27/finetuned_enhanced_low/*_noisy.wav
! rm -rf ../../../drive/MyDrive/IDLProject/april27/finetuned_enhanced_auto/*_noisy.wav

# Pip Installations for Evaluating Metrics

In [5]:
! pip install git+https://github.com/aliutkus/speechmetrics#egg=speechmetrics[cpu] --quiet

import speechmetrics

DEPRECATION: git+https://github.com/aliutkus/speechmetrics#egg=speechmetrics[cpu] contains an egg fragment with a non-PEP 508 name pip 25.0 will enforce this behaviour change. A possible replacement is to use the req @ url syntax, and remove the egg fragment. Discussion can be found at https://github.com/pypa/pip/issues/11617
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 63.4 MB/s eta 0:00:00
     \ 59.4 MB 120.1 MB/s 0:00:04
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 963.5/963.5 kB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 15.3 MB/s eta 0:00:00


In [6]:
! pip3 install https://github.com/schmiph2/pysepm/archive/master.zip --quiet

import pysepm

     \ 1.8 MB 15.3 MB/s 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     - 39.3 kB 6.1 MB/s 0:00:00
  Preparing metadata (setup.py) ... done
     - 223.1 kB 5.9 MB/s 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done


In [7]:
! pip3 install pystoi --quiet

import pystoi

# Metrics Calculations

In [ ]:
!pwd

/content/TAPLoss/Demucs/denoiser


In [9]:
import librosa

# n = 0

Y01, Y02, Y03 = 0, 0, 0
Y04, Y05, Y06 = 0, 0, 0
Y07, Y08, Y09 = 0, 0, 0
Y10, Y11, Y12 = 0, 0, 0
Y13, Y14, Y15 = 0, 0, 0
Y16, Y17, Y18 = 0, 0, 0
Y19, Y20, Y23 = 0, 0, 0
Y24, Y25, Y26 = 0, 0, 0

# print(os.listdir("drive"))
for file_clean in os.listdir("../../../drive/MyDrive/IDLProject/april27/clean/src_clean"): 
  for file_noisy in os.listdir("../../../drive/MyDrive/IDLProject/april27/fullsubnet/finetuned_enhanced_low"):
    if (file_clean.split("_", 1)[1].split(".",1)[0]) in file_noisy:
      break

  print(file_clean, file_noisy)
  clean_url = "../../../drive/MyDrive/IDLProject/april27/clean/src_clean/" + file_clean
  noisy_url = "../../../drive/MyDrive/IDLProject/april27/fullsubnet/finetuned_enhanced_low/" + file_noisy
  # os.system("wget -O clean.wav " + clean_url)
  # os.system("wget -O noisy.wav " + noisy_url)
  clean = librosa.load(clean_url, sr=16000)[0]
  noisy  = librosa.load(noisy_url, sr=16000)[0]
#   ### PYSEPM
  Y01_temp, Y02_temp, Y03_temp = pysepm.composite(clean, noisy, 16000)
  Y04_temp, Y05_temp, Y06_temp = pysepm.csii(clean, noisy, 16000)
  Y07_temp           = pysepm.fwSNRseg(clean, noisy, 16000)
  Y08_temp          = pysepm.SNRseg(clean, noisy, 16000)
  Y09_temp          = pysepm.llr(clean, noisy, 16000)
  Y10_temp           = pysepm.ncm(clean, noisy, 16000)
  Y11_temp           = pysepm.wss(clean, noisy, 16000)

#   ### SPEECHMETRICS
  Y12_temp, Y13_temp, Y14_temp, Y15_temp, Y16_temp, Y17_temp, Y18_temp = speechmetrics.load(
      'relative', window=None)(noisy, clean, rate=16000).values()
  

  Y19_temp, Y20_temp = speechmetrics.load(
      'absolute', window=None)(noisy, rate=16000).values()
  
  Y23 += pystoi.stoi(clean, noisy, fs_sig=16000, extended=False)
  Y24 += pystoi.stoi(clean, noisy, fs_sig=16000, extended=True)

  Y01 += Y01_temp
  Y02 += Y02_temp
  Y03 += Y03_temp
  Y04 += Y04_temp
  Y05 += Y05_temp
  Y06 += Y06_temp
  Y07 += Y07_temp
  Y08 += Y08_temp
  Y09 += Y09_temp
  Y10 += Y10_temp
  Y11 += Y11_temp
  Y12 += Y12_temp.item()
  Y13 += Y13_temp.item()
  Y14 += Y14_temp.item()
  Y15 += Y15_temp
  Y16 += Y16_temp
  Y17 += Y17_temp
  Y18 += Y18_temp
  Y19 += Y19_temp.item()
  Y20 += Y20_temp.item()


# ### PRINT

print(
    "\n",
    "Composite ", Y01/150, "    ", Y02/150, "    ", Y03/150, "\n",
    "CSII      ", Y04/150, "    ", Y05/150, "    ", Y06/150, "\n",
    "fwSNRseg  ", Y07/150, "\n",
    "SNRseg    ", Y08/150, "\n",
    "LLR       ", Y09/150, "\n",
    "NCM       ", Y10/150, "\n",
    "WSS       ", Y11/150,
    sep="")

print(
    "\n",
    "SDR     ", Y12/150, "\n",
    "ISR     ", Y13/150, "\n",
    "SAR     ", Y14/150, "\n",
    "NB_PESQ ", Y15/150, "\n",
    "PESQ    ", Y16/150, "\n", #important
    "SISDR   ", Y17/150, "\n",
    "STOI    ", Y18/150, "\n", #important
    "MOSNET  ", Y19/150, "\n",
    "SRMR    ", Y20/150,
    sep="")

# ### PYSTOI
print(
    "STOI   ", Y23/150, "\n",
    "ESTOI  ", Y24/150,
    sep=""
)

clean_fileid_105.wav clnsp591_clatter_noise_166453__zabuhailo__metalworking-hammer-5_5_snr9_tl-27_fileid_105.wav


/usr/local/lib/python3.10/dist-packages/pysepm/intelligibilityMeasures.py:79: RuntimeWarning: divide by zero encountered in log10
  rms_db = 20*np.log10(rms_seg/rms_all);


Loaded  speechmetrics.relative.bsseval
Loaded  speechmetrics.relative.nb_pesq
Loaded  speechmetrics.relative.pesq
Loaded  speechmetrics.relative.sisdr
Loaded  speechmetrics.relative.stoi
Loaded  speechmetrics.absolute.mosnet
Loaded  speechmetrics.absolute.srmr
clean_fileid_101.wav clnsp198_bus_56903_0_snr0_tl-32_fileid_101.wav


/usr/local/lib/python3.10/dist-packages/pysepm/intelligibilityMeasures.py:79: RuntimeWarning: divide by zero encountered in log10
  rms_db = 20*np.log10(rms_seg/rms_all);
/usr/local/lib/python3.10/dist-packages/pysepm/intelligibilityMeasures.py:142: RuntimeWarning: invalid value encountered in true_divide
  SDR = processed_energy/de_processed_energy;# Eq 13 in Kates (2005)


Loaded  speechmetrics.relative.bsseval
Loaded  speechmetrics.relative.nb_pesq
Loaded  speechmetrics.relative.pesq
Loaded  speechmetrics.relative.sisdr
Loaded  speechmetrics.relative.stoi
Loaded  speechmetrics.absolute.mosnet
Loaded  speechmetrics.absolute.srmr
clean_fileid_110.wav clnsp305_baby_416655_0_snr11_tl-30_fileid_110.wav


...

Loaded  speechmetrics.relative.bsseval
Loaded  speechmetrics.relative.nb_pesq
Loaded  speechmetrics.relative.pesq
Loaded  speechmetrics.relative.sisdr
Loaded  speechmetrics.relative.stoi
Loaded  speechmetrics.absolute.mosnet
Loaded  speechmetrics.absolute.srmr
clean_fileid_50.wav clnsp22_breath_21983_0_snr1_tl-18_fileid_50.wav


/usr/local/lib/python3.10/dist-packages/pysepm/intelligibilityMeasures.py:79: RuntimeWarning: divide by zero encountered in log10
  rms_db = 20*np.log10(rms_seg/rms_all);


Loaded  speechmetrics.relative.bsseval
Loaded  speechmetrics.relative.nb_pesq
Loaded  speechmetrics.relative.pesq
Loaded  speechmetrics.relative.sisdr
Loaded  speechmetrics.relative.stoi
Loaded  speechmetrics.absolute.mosnet
Loaded  speechmetrics.absolute.srmr
clean_fileid_8.wav clnsp16_traffic_248091_1_snr11_tl-22_fileid_88.wav


/usr/local/lib/python3.10/dist-packages/pysepm/intelligibilityMeasures.py:79: RuntimeWarning: divide by zero encountered in log10
  rms_db = 20*np.log10(rms_seg/rms_all);


Loaded  speechmetrics.relative.bsseval
Loaded  speechmetrics.relative.nb_pesq
Loaded  speechmetrics.relative.pesq
Loaded  speechmetrics.relative.sisdr
Loaded  speechmetrics.relative.stoi
Loaded  speechmetrics.absolute.mosnet
Loaded  speechmetrics.absolute.srmr

Composite 1.4275153803341212    1.75555649227376    1.3912522208344686
CSII      nan    nan    nan
fwSNRseg  1.694301965320925
SNRseg    -1.4516408857793244
LLR       1.6014195074364044
NCM       0.30715320910873023
WSS       79.05494150312411

SDR     0.15235948978671304
ISR     0.7849246965735459
SAR     -9.111338976332782
NB_PESQ 2.2346344459056855
PESQ    1.5832022937138874
SISDR   -16.05398067443942
STOI    0.47237377050091356
MOSNET  2.8687024799982708
SRMR    9.476652393345804
STOI   0.47237377050091356
ESTOI  0.334326303128455


In [ ]:
# ! mkdir src_clean
# ! pwd
# # Put src_noisy.zip into content folder 
# ! unzip "/content/drive/MyDrive/11785 DL Project/src_clean.zip" -d /content/

n = len(os.listdir("/content/drive/MyDrive/IDLProject/april27/clean/src_clean"))
assert(len(os.listdir("/content/drive/MyDrive/IDLProject/april27/clean/src_clean")) == len(os.listdir("/content/drive/MyDrive/IDLProject/april27/finetuned_enhanced_low")))
print(n)

150
